In [ ]:
from __future__ import print_function, division
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import gdown
from zipfile import ZipFile
import imageio
import tensorflow_datasets as tfds
import sys
import random
from PIL import Image
import matplotlib.pyplot as plt
from math import floor    
from keras.layers import Conv2D, LeakyReLU, BatchNormalization, Dense, AveragePooling2D, GaussianNoise
from keras.layers import Reshape, UpSampling2D, Activation, Dropout, Flatten, Conv2DTranspose
from keras.models import model_from_json, Sequential

In [ ]:
from google.colab import drive
import os
drive.mount('/content/GDrive')


path = "/content/GDrive/MyDrive/Plants_Images/Plants" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

Drive already mounted at /content/GDrive; to attempt to forcibly remount, call drive.mount("/content/GDrive", force_remount=True).


https://keras.io/examples/generative/dcgan_overriding_train_step/

In [ ]:
Plants = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/GDrive/MyDrive/Plants_Images/Plants",
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode="rgb",
    batch_size=4,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 13921 files belonging to 1 classes.


In [ ]:
dataset = Plants.map(lambda x: x / 255.0)

In [ ]:
discriminator = Sequential(name= "discriminator")
      
#add Gaussian noise to prevent Discriminator overfitting
discriminator.add(GaussianNoise(0.2, input_shape = [256, 256, 3]))

#256x256x3 Image
discriminator.add(Conv2D(filters = 8, kernel_size = 3, padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(AveragePooling2D())

#128x128x8
discriminator.add(Conv2D(filters = 16, kernel_size = 3, padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(AveragePooling2D())

#64x64x16
discriminator.add(Conv2D(filters = 32, kernel_size = 3, padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(AveragePooling2D())

#32x32x32
discriminator.add(Conv2D(filters = 64, kernel_size = 3, padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(AveragePooling2D())

#16x16x64
discriminator.add(Conv2D(filters = 128, kernel_size = 3, padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(AveragePooling2D())

#8x8x128
discriminator.add(Conv2D(filters = 256, kernel_size = 3, padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(AveragePooling2D())

#4x4x256
discriminator.add(Flatten())

#256
discriminator.add(Dense(128))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(1, activation = 'sigmoid'))
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gaussian_noise (GaussianNoi  (None, 256, 256, 3)      0         
 se)                                                             
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 8)       224       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256, 256, 8)       0         
                                                                 
 average_pooling2d (AverageP  (None, 128, 128, 8)      0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 16)      1168      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128, 128, 16)    

In [ ]:
latent_dim = 4096

generator = Sequential(name="generator")
        
generator.add(Reshape(target_shape = [1, 1, latent_dim], input_shape = [latent_dim]))

#1x1x4096 
generator.add(Conv2DTranspose(filters = 256, kernel_size = 4))
generator.add(LeakyReLU(0.2))

#4x4x256
generator.add(Conv2D(filters = 256, kernel_size = 4, padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D())

#8x8x256 
generator.add(Conv2D(filters = 128, kernel_size = 4, padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D())

#16x16x128
generator.add(Conv2D(filters = 64, kernel_size = 3, padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D())

#32x32x64
generator.add(Conv2D(filters = 32, kernel_size = 3, padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D())

#64x64x32
generator.add(Conv2D(filters = 16, kernel_size = 3, padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D())

#128x128x16
generator.add(Conv2D(filters = 8, kernel_size = 3, padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D())

#256x256x8
generator.add(Conv2D(filters = 3, kernel_size = 3, padding = 'same'))
generator.add(Activation('sigmoid'))

generator.summary()


Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 1, 1, 4096)        0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 4, 4, 256)        16777472  
 nspose)                                                         
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 4, 4, 256)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 256)         1048832   
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 4, 4, 256)         0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 8, 8, 256)        0         
 )                                                       

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=72):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("/content/GDrive/MyDrive/Plants_Images/Generated_Test/generated_img_%03d_%d.png" % (epoch, i))

In [ ]:
epochs = 100  # In practice, use ~100 epochs

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)

Epoch 1/100
3481/3481 [==============================] - 79s 22ms/step - d_loss: -0.0501 - g_loss: 38.2823
Epoch 2/100
3481/3481 [==============================] - 76s 22ms/step - d_loss: -14.6788 - g_loss: 1722.9987
Epoch 3/100
3481/3481 [==============================] - 76s 22ms/step - d_loss: -81.0783 - g_loss: 9805.1592
Epoch 4/100
3481/3481 [==============================] - 76s 22ms/step - d_loss: -618.5592 - g_loss: 67229.4453
Epoch 5/100
3481/3481 [==============================] - 76s 22ms/step - d_loss: 347.8682 - g_loss: 370041.3750
Epoch 6/100
3481/3481 [==============================] - 76s 22ms/step - d_loss: 2741.9548 - g_loss: 105.9399
Epoch 7/100
3481/3481 [==============================] - 76s 22ms/step - d_loss: 173.6590 - g_loss: 24.2863
Epoch 8/100
3481/3481 [==============================] - 76s 22ms/step - d_loss: 4.3138 - g_loss: 124.1083
Epoch 9/100
3481/3481 [==============================] - 76s 22ms/step - d_loss: 15.5659 - g_loss: 526.9484
Epoch 10/100
348